# DQN-Agent for our own Env

**imports**

In [2]:
from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Permute, Dense, Convolution2D, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.callbacks import ModelIntervalCheckpoint, FileLogger
from rl.core import Processor

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**setup env**

In [3]:
env = gym.make("snake:snake-v0")
nb_actions = env.action_space.n
nb_actions

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\Users\tobia\.conda\envs\ai\lib\site-packages\gym\spaces\box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
c:\Users\tobia\.conda\envs\ai\lib\site-packages\gym\utils\passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float16. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
c:\Users\tobia\.conda\envs\ai\lib\site-packages\gym\utils\passive_env_checker.

4

**image processing**

In [6]:
IMG_SHAPE = (84, 84)    # downsampling
WINDOW_LENGTH = 4 
INPUT_SHAPE = (WINDOW_LENGTH, *IMG_SHAPE)    

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
class ImageProcessor(Processor):
    def process_observation(self, observation):
        img = Image.fromarray(observation)
        img = img.resize(IMG_SHAPE)
        img = img.convert("L")
        img = np.array(img)
        return img.astype("uint8")

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1.0, 1.0)

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**create model**

In [8]:
model = Sequential()

model.add(Permute((2,3,1), input_shape=INPUT_SHAPE))

model.add(Convolution2D(filters=32, kernel_size=(8,8), strides=(4,4), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(filters=64, kernel_size=(4,4), strides=(2,2), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(filters=64, kernel_size=(3,3), strides=(1,1), kernel_initializer='he_normal'))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(nb_actions))
model.add(Activation('linear'))

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
activation (Activation)      (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_1 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 64)          0

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**creating agent**

In [10]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),
                              attr='eps',
                              value_max=1.0,
                              value_min=0.1,
                              value_test=0.05,
                              nb_steps=1000000)

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
processor = ImageProcessor()

In [15]:
dqn = DQNAgent(model=model, policy=policy, memory=memory, processor=processor,
               nb_actions=nb_actions, nb_steps_warmup=50000, gamma=0.99, target_model_update=10000,
               train_interval=4, delta_clip=1)

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
dqn.compile(optimizer=Adam(lr=25e-5), metrics=['mae'])

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\Users\tobia\.conda\envs\ai\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [17]:
weights_filename = "dqn_snake_weights.h5f"
checkpoint_filename = "dqn_snake_weights_checkpoint.h5f"
checkpoint_callbacks = ModelIntervalCheckpoint(checkpoint_filename, interval=10000)

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
dqn.fit(env=env, nb_steps=1500000, callbacks=[checkpoint_callbacks], log_interval=100000, visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

**load finished weights**

In [18]:
model.load_weights("../COURSE_NOTEBOOKS/10-Custom-RL/snake_weights/dqn_snake_weights_1200000.h5f")

memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

policy = EpsGreedyQPolicy()

processor = ImageProcessor()

dqn = DQNAgent(model=model, policy=policy, memory=memory, processor=processor,
               nb_actions=nb_actions, nb_steps_warmup=50000, gamma=0.99, target_model_update=10000,
               train_interval=4, delta_clip=1)

dqn.compile(optimizer=Adam(lr=25e-5), metrics=['mae'])

C:\Users\tobia\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\Users\tobia\.conda\envs\ai\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


**try out**

In [33]:
env.sleep = 0.5

In [34]:
dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
Episode 1: reward: -1.000, steps: 35
